In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df = r"/submission_sample.csv"
df = pd.read_csv(df)
df.head()


,srch_id,prop_id
0,1,3180
1,1,5543
2,1,14142
3,1,22393
4,1,24194


In [12]:
train = r"/preprocessed_train_data1.csv"
df_train = pd.read_csv(train)
df_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,click_bool,gross_bookings_usd,booking_bool,day_of_week,month,hour_of_day,is_weekend,same_country,delta_star_rating,delta_price_usd
0,1,2013-04-04 08:32:15,12,187,3.374334,176.022659,219,893,3,3.5,...,0.0,NaN,0.0,3.0,4.0,8.0,0.0,0.0,-0.374334,-71.252659
1,1,2013-04-04 08:32:15,12,187,3.374334,176.022659,219,10404,4,4.0,...,0.0,NaN,0.0,3.0,4.0,8.0,0.0,0.0,0.625666,-5.282659
2,1,2013-04-04 08:32:15,12,187,3.374334,176.022659,219,21315,3,4.5,...,0.0,NaN,0.0,3.0,4.0,8.0,0.0,0.0,-0.374334,3.777341
3,1,2013-04-04 08:32:15,12,187,3.374334,176.022659,219,27348,2,4.0,...,0.0,NaN,0.0,3.0,4.0,8.0,0.0,0.0,-1.374334,426.747341
4,1,2013-04-04 08:32:15,12,187,3.374334,176.022659,219,29604,4,3.5,...,0.0,NaN,0.0,3.0,4.0,8.0,0.0,0.0,0.625666,-32.442659


In [13]:
df_train[df_train['price_usd']>5000].count()

srch_id                        556
date_time                      556
site_id                        556
visitor_location_country_id    556
visitor_hist_starrating        556
                              ... 
hour_of_day                    556
is_weekend                     556
same_country                   556
delta_star_rating              556
delta_price_usd                556
Length: 61, dtype: int64

In [14]:
test = r"/test_set_VU_DM.csv"
df_test = pd.read_csv(test)
df_test.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def create_submission(model1, model2, Test_data, proba = False):
    import time
    start = time.time()

    Test = Test_data
    #predictions
    print("~ Making predictions")
    if proba == True:
        clickpreds = model1.predict_proba(Test.drop(['srch_id','prop_id'], axis = 1))
        bookpreds = model2.predict_proba(Test.drop(['srch_id','prop_id'], axis = 1))

        Test['click_preds'] = pd.DataFrame(clickpreds)[1].values
        Test['booking_preds'] = pd.DataFrame(bookpreds)[1].values
    else:
        clickpreds = model1.predict(Test.drop(['srch_id','prop_id'], axis = 1))
        bookpreds = model2.predict(Test.drop(['srch_id','prop_id'], axis = 1))

        Test['click_preds'] = clickpreds
        Test['booking_preds'] = bookpreds

    #Grab 4 columns from Test
    sub = Test[['srch_id','prop_id','click_preds','booking_preds']]
    #Weight booking column values

    if proba == True:
        sub['booking_preds'] = sub['booking_preds'].apply(lambda x: x*2 if x > 0 else x)
    else:
        sub['booking_preds'] = sub['booking_preds'].apply(lambda x: x+1 if x > 0 else x)
    #Total click & booking values, remove columns
    sub['total'] = sub['click_preds']+sub['booking_preds']
    sub.drop(['click_preds', 'booking_preds'], axis = 1, inplace = True)

    #Sort chunks of srch ids by 'total' value
    print(f"~ Sorting chunks of srch_ids: {(time.time()-start):.2f}s have passed")
    ids = sub['srch_id'].unique()
    sub1 = pd.DataFrame()
    k=0

    srch_id = []
    prop_id = []
    total = []
    for item in ids:

        df = sub[sub['srch_id'] == item].sort_values('total', ascending = False)
        srch_id.append(df['srch_id'].values)
        prop_id.append(df['prop_id'].values)
        total.append(df['total'].values)

        k+=1
        if not k%10000:
            print(f"{k}/{len(ids)} srch_ids, time elapsed: {(time.time() - start):.2f}s")

    #Unpack list of arrays
    print(f"~ Unpacking lists: {(time.time() - start):.2f}s elapsed")
    _srch_id = []
    for i in range(0,len(srch_id)):
        for k in range(0,len(srch_id[i])):
            _srch_id.append(srch_id[i][k])
    _prop_id = []
    for i in range(0,len(prop_id)):
        for k in range(0,len(prop_id[i])):
            _prop_id.append(prop_id[i][k])
    _total = []
    for i in range(0,len(total)):
        for k in range(0,len(total[i])):
            _total.append(total[i][k])

    #Create Dataframe from unpacked lists
    print(f"~ Creating Dataframe: {(time.time() - start):.2f}s elapsed")
    sub2 = pd.DataFrame(zip(_srch_id, _prop_id, _total),
               columns =['srch_id', 'prop_id','total'])

    sub1 = sub2[['srch_id','prop_id']]

    print(f"~ Finished. Total time: {(time.time() - start):.2f}")
    return(sub1, sub2)




In [17]:
X_train = df_train.drop(['srch_id', 'prop_id','click_bool', 'booking_bool'], axis = 1)
Y1_train = df_train['click_bool']
Y2_train = df_train['booking_bool']

In [18]:
LGBMclick = lgb.LGBMClassifier(scale_pos_weight = 12, n_estimators = 76)
LGBMbook = lgb.LGBMClassifier(scale_pos_weight = 12, n_estimators = 78)

In [19]:
LGBMclick.fit(X_train, Y1_train)
LGBMbook.fit(X_train, Y2_train)

ValueError: Input contains NaN.

In [20]:
sorted_idx = LGBMclick.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], LGBMclick.feature_importances_[sorted_idx])
plt.xlabel("LightGBM Click Feature Importance")
plt.title("LightGBM Click Model: Features by Importance")

NotFittedError: No feature_importances found. Need to call fit beforehand.

In [21]:
sorted_idx = LGBMbook.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], LGBMbook.feature_importances_[sorted_idx])
plt.xlabel("Importance")
plt.title("LightGBM Booking Model: Features by Importance")

NotFittedError: No feature_importances found. Need to call fit beforehand.

In [23]:
results = create_submission(LGBMclick,LGBMbook,df_test, proba = True)

~ Making predictions


NotFittedError: Estimator not fitted, call fit before exploiting the model.

In [24]:
results[1]

NameError: name 'results' is not defined

In [25]:
results[0]

NameError: name 'results' is not defined

In [26]:
results[0].to_csv('/Kaggle_Submission.csv',index=False)

NameError: name 'results' is not defined

In [27]:
df_train['estimated_position'].median()

KeyError: 'estimated_position'

In [28]:
import seaborn as sns

In [29]:
print(len(results[0]))

NameError: name 'results' is not defined